In [1]:
# Requisitos
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Dense, LeakyReLU
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Cargar el dataset
(X_train, _), (_,_) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.0
X_train = X_train.reshape(-1,28,28,1).astype('float32')

In [3]:
#tamaño selespacio latente, ruido de entrada para el generador
latent_dim = 100

In [4]:
# crear generador
def bull_generator():
  model = Sequential([Dense( 256, input_dim=latent_dim),LeakyReLU(negative_slope = 0.2),BatchNormalization(), Dense(512),LeakyReLU(negative_slope=0.2),BatchNormalization(), Dense(1024), LeakyReLU(negative_slope= 0.2),BatchNormalization(), Dense(28*28*1, activation='tanh'), Reshape((28,28,1))])
  return model
  #model.add(layers.Dense(7*7*256, input_dim=latent_dim))
  #model.add

In [5]:
# Crear el discriminador
def build_discriminator():
    model = Sequential([Flatten(input_shape=(28,28,1)), Dense(512), LeakyReLU(negative_slope=0.2), Dense(1, activation='sigmoid')])
    return model

In [6]:
# Configurarel modelo
discriminador = build_discriminator()
discriminador.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5), metrics=['accuracy'])

C:\Users\medin\mp-env\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
#Configurar el generador
generador = bull_generator()
generador.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

C:\Users\medin\mp-env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'BatchNormalization' is not defined

In [ ]:
#Crear la Gan
gan_input = tf.keras.Input(shape=(latent_dim,))
gan_output = discriminador(generador(gan_input))
gan = tf.keras.Model(gan_input, gan_output)
discriminador.trainable = False

In [ ]:
# Compile gan
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [ ]:
# Funcion para entrenar la GAN
def train_gan(epochs=10000, batch_size=128):
    batch_count = X_train.shape[0]
    
    for epoch in range(epochs):
        for _ in range(batch_count):
            # Entrenar el discriminador
            noise = np.random.normal(0,1,size=(batch_size, latent_dim))
            generated_images = generador.predict(noise)
            real_images = X_train[np.random.randint(0, X_train.shape[0],batch_size)]

            X = np.concatenate([real_images, generated_images])
            y = np.concatenate([np.ones((batch_size,1))], [np.zeros((batch_size,1))])

            d_loss = discriminator.train_on_batch(X,y)

            # Entrenar el generador por medio de la GAN
            g_loss = gan.train_on_batch(noise, np.ones((batch_size,1)))

        if epoch % 1000 == 0:
            print(f'Epoch: {epoch}, Pérdida Discriminador: {d_loss[0]}, Pérdida Generador: {g_loss}')
            generate_and_save_images(generador, epoch)

In [ ]:
# Funcion para generar y guardar imagenes
def generate_and_save_images(model, epoch, ejemplos=16, dim=(4,4), figsize=(10,10)):
    noise = np.random.normal(0,1,size=(ejemplos, latent_dim))
    generated_images = model.predict(noise)
    generated_images = generated_images.reshape(ejemplos, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()